# Required imports

In [1]:
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.embeddings.llamafile import LlamafileEmbedding
from llama_index.llms.llamafile import Llamafile

# for reading csv
from llama_index.readers.file import CSVReader
from pathlib import Path

from IPython.display import Markdown, display

# Basic client setup

In [2]:
# configure LlamaIndex to use the LLaMAFile
# create an embedding model that utilizes the LLaMAFile
Settings.embed_model = LlamafileEmbedding(base_url="http://localhost:8080")
# Set up the Llama model as the LLM component of LlamaIndex
Settings.llm = Llamafile(base_url="http://localhost:8080", temperature=0, seed=0)

# Code Samples

## Using Default Prompts

In [5]:
# load local pdf docs
# the `load_data` method loads the documents from the directory and returns them as a list. 
local_reader = SimpleDirectoryReader(input_dir='../../../../../_001.Data/sample-pdf/')
docs = local_reader.load_data(show_progress=True)

Loading files: 100%|█████████████████████████████████████████████████| 1/1 [00:00<00:00,  9.65file/s]


In [4]:
# load local csv files
file = Path('../../../../../_001.Data/FinancialsSampleData.csv')
csv_reader = CSVReader()
csvs = csv_reader.load_data(file=file)

In [6]:
# create an index to store vector representations of the loaded documents. 
# the `from_documents` method builds the index from the provided list of documents.
index_pdf = VectorStoreIndex.from_documents(docs)
index_csv = VectorStoreIndex.from_documents(csvs)

In [7]:
# convert the index into a query engine that can handle queries and query the index
query_engine_pdf = index_pdf.as_query_engine()
query_engine_csv = index_csv.as_query_engine()

In [8]:
# use the query engine to retrieve relevant information from the index
query = "What is the main topic of the document?"
response = query_engine_pdf.query(query)
print(response)

The main topic of the document is the context information provided, which includes information about the given context, the rules to follow when answering a question, and the context information provided in the document.</s>


In [9]:
# query the csv index
query = "What are the currencies being used?"
response = query_engine_csv.query(query)
print(response)

The currencies being used in the given context are USD, EUR, GBP, AUD, CAD, JPY, and CHF.</s>


## Using Custom Prompts

### [Completion Prompts](https://docs.llamaindex.ai/en/stable/examples/customization/prompts/completion_prompts/)

In [10]:
from llama_index.core import PromptTemplate

text_qa_template_str = (
    "Context information is"
    " below.\n---------------------\n{context_str}\n---------------------\nUsing"
    " both the context information and also using your own knowledge, answer"
    " the question: {query_str}\nIf the context isn't helpful, you can also"
    " answer the question on your own.\n"
)
text_qa_template = PromptTemplate(text_qa_template_str)

refine_template_str = (
    "The original question is as follows: {query_str}\nWe have provided an"
    " existing answer: {existing_answer}\nWe have the opportunity to refine"
    " the existing answer (only if needed) with some more context"
    " below.\n------------\n{context_msg}\n------------\nUsing both the new"
    " context and your own knowledge, update or repeat the existing answer.\n"
)
refine_template = PromptTemplate(refine_template_str)

In [11]:
print(
    index_pdf.as_query_engine(
        text_qa_template=text_qa_template,
        refine_template=refine_template,
    ).query("What is the main topic of the document?")
)

The main topic of the document is the topic of mushroom-inoculated bricks being used to clean up contaminated soil in Los Alamos, New Mexico.</s>


### [Chat Prompts](https://docs.llamaindex.ai/en/stable/examples/customization/prompts/chat_prompts/)

In [12]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

qa_prompt_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

refine_prompt_str = (
    "We have the opportunity to refine the original answer "
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "Given the new context, refine the original answer to better "
    "answer the question: {query_str}. "
    "If the context isn't useful, output the original answer again.\n"
    "Original Answer: {existing_answer}"
)

In [13]:
# Text QA Prompt
chat_text_qa_msgs = [
    (
        "system",
        "Always answer the question, even if the context isn't helpful.",
    ),
    ("user", qa_prompt_str),
]
text_qa_template = ChatPromptTemplate.from_messages(chat_text_qa_msgs)

# Refine Prompt
chat_refine_msgs = [
    (
        "system",
        "Always answer the question, even if the context isn't helpful.",
    ),
    ("user", refine_prompt_str),
]
refine_template = ChatPromptTemplate.from_messages(chat_refine_msgs)

In [14]:
print(
    index_pdf.as_query_engine(
        text_qa_template=text_qa_template,
        refine_template=refine_template,
    ).query("What is the main topic of the document?")
)

The main topic of the document is the issue of contaminated soil and water in Los Alamos, New Mexico, and the efforts to address it by using mushrooms as a natural remediation method.</s>
